## This notebook is part of the Apache Spark training delivered by CERN-IT
### Spark SQL Hands-On Lab
Contact: Luca.Canali@cern.ch

Run this notebook from Jupyter with Python kernel
- When using on CERN SWAN, do not attach the notebook to a Spark cluster, but rather run locally on the SWAN container
- If running this outside CERN SWAN, please make sure to have PySpark installed: `pip install pyspark`

## Examples datasets
The following examples use sample data provided in the repository.  
We will use the movielens dataset from Kaggle, credits: https://www.kaggle.com/grouplens/movielens-20m-dataset

In [ ]:
# Create Spark Session, you need this to work with Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
        .appName("My spark example app")  \
        .master("local[*]") \
        .config("spark.driver.memory","8g") \
        .config("spark.ui.showConsoleProgress", "false") \
        .getOrCreate()

In [ ]:
spark

In [ ]:
# sets the path to the directory with datafiles
PATH = "../data/"

In [ ]:
ratings = spark.read.option("header","true").csv(PATH + "ratings1.csv.gz")

In [ ]:
ratings.show(5)

In [ ]:
ratings.printSchema()

In [ ]:
# infer schema needs to go through the data to estimate the schema, this takes time

%time ratings = spark.read.option("header","true").option("inferSchema", "true").csv(PATH + "ratings1.csv.gz")

# note ratings*.csv.gz will read rating1.csv.gz and ratings2.csv.gz, more data, however slower to run
# spark.read.option("header","true").option("inferSchema", "true").csv(PATH + "ratings*.csv.gz")

In [ ]:
ratings.printSchema()

In [ ]:
# movielens dataset
movies = spark.read.option("header","true").option("inferSchema", "true").csv(PATH + "movies.csv.gz")

In [ ]:
movies.show(5, False)

In [ ]:
movies.printSchema()

In [ ]:
tags = spark.read.option("header","true").option("inferSchema", "true").csv(PATH + "tags.csv.gz")

In [ ]:
tags.show(5)

#### Register the dataframes as Spark Temporary Views

In [ ]:
ratings.createOrReplaceTempView("ratings")
movies.createOrReplaceTempView("movies")
tags.createOrReplaceTempView("tags")

In [ ]:
# note what happens when we query a table in a csv file with a filter
spark.sql("select movieId, title from movies where movieId=1").explain()

In [ ]:
# cache the tables, to improve the performance of the rest of the queries in the notebook
# note: default caching level is MEMORY_AND_DISK (i.e. caching in memory if enough heap is available)
# note: caching is lazily executed, so a count() action is added to make the operation happen
# this operation may take a couple of minutes

r = ratings.cache().count()
m = movies.cache().count()
t = tags.cache().count()

In [ ]:
print(f"Num ratings = {r}\nNum tags = {t}\nNum movies = {m}")

### SQL Queries on the loaded tables
#### 1) How many movies produced per year?

In [ ]:
# Add the column Year to "movies"



In [ ]:
# retrieve the number of movies per year


In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
plt.style.use('seaborn-darkgrid')

In [ ]:
# prolt the results
m_yr.plot(x='year',y='count',kind='line', title='Movies per year');

#### 2) Top movies by number of ratings

In [ ]:
# A query to perform a join operation between movies and ratings
# Find the highest rated movies



#### What happened in the background? How did the join happen?
#### How is the query executed? Can we get more information?

#### 3) Highly rated movies
Find the top 5 highly rated movies

#### Drill down on the top entries:
   - How many reviews contributed to this rating?

Lets only take in account movies that have more than 100 reviews

#### 4) Find the top rated movie of every year since 2000

In [ ]:
# End the Spark application
spark.stop()